In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "cfbd.settings")
import django
django.setup()

import pandas as pd
import numpy as np

from main import models

In [2]:
rng = np.random.default_rng()

## Input Data

In [3]:
schools = pd.read_csv("recruitingProbabilities.csv")
schools["Total"] = 0
schools

,Team,5_Raw,4_Raw,3_Raw,5_Exp,4_Exp,3_Exp,5_Prob,4_Prob,3_Prob,Total
0,Alabama,11,58,13,13.006555,57.299817,14.138378,0.116176,0.044346,0.002444,0
1,Georgia,13,47,25,12.143640,48.912875,20.901967,0.108468,0.037855,0.003613,0
2,LSU,14,45,25,10.386767,47.650624,27.321595,0.092775,0.036879,0.004722,0
3,Oklahoma,6,41,31,8.670747,46.439520,32.031843,0.077448,0.035941,0.005536,0
4,Ohio State,7,44,32,6.979414,45.228630,35.907093,0.062341,0.035004,0.006206,0
...,...,...,...,...,...,...,...,...,...,...,...
125,New Mexico State,0,0,12,0.000000,0.524401,8.854138,0.000000,0.000406,0.001530,0
126,Texas State,0,1,9,0.000000,0.516536,7.981228,0.000000,0.000400,0.001379,0
127,UL-Monroe,0,0,5,0.000000,0.508671,7.108318,0.000000,0.000394,0.001229,0
128,Coastal Carolina,0,3,7,0.000000,0.500806,6.235408,0.000000,0.000388,0.001078,0


In [4]:
players = pd.read_csv("players_evaluated.csv")
players["School"] = ""
players["Scholarship"] = False
players.index += 1
players

,First,Last,Height,Weight,T_Strength,T_Athleticism,T_Dexterity,T_IQ,T_Personality,T_Arm,...,S_ReadOff,S_KickPow,S_KickAcc,S_PuntPow,S_PuntAcc,Position,P_BEST,Stars,School,Scholarship
1,Andre,Brown,77.785520,370.251049,96.205956,38.034630,31.360394,33.230310,49.318560,43.095424,...,46.534546,58.241582,71.410036,58.880498,65.581198,OL,108.370395,5,,False
2,Alijah,Harris,76.678068,299.611678,90.896314,49.165386,29.255212,74.229956,43.171610,42.392602,...,69.298571,39.977992,45.391858,42.082889,53.688686,DT,103.209994,5,,False
3,Weston,Jackson,78.264296,344.850574,94.834343,34.736893,20.231105,57.028000,48.219405,31.305556,...,58.903932,36.826000,37.802871,40.034481,36.233092,DT,102.063971,5,,False
4,Michael,Finley,80.998230,370.025541,80.890109,28.750672,23.364785,22.856387,50.052748,65.370502,...,32.124837,72.085488,64.610865,74.140242,49.754739,OL,101.260238,5,,False
5,Trevor,Kennedy,78.825032,337.558931,92.124447,29.626324,25.696194,41.070697,57.389817,46.083700,...,45.194881,35.207365,28.020470,36.220736,34.622699,OL,101.244444,5,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11186,Alexander,Taylor,73.069718,192.576496,12.249627,41.622411,81.585046,51.615538,53.051356,17.944516,...,54.973832,27.942084,36.691033,32.014368,42.943702,WR,38.044633,2,,False
11187,Ryan,Anderson,73.656113,191.024157,32.852587,48.698064,52.321087,44.950419,43.752979,33.823738,...,49.772971,22.623272,31.602734,24.885373,45.944774,EDGE,37.891980,2,,False
11188,Jose,Taylor,70.901010,198.336853,26.064285,44.826046,63.228263,55.632430,51.818022,15.915664,...,57.518376,31.744278,35.300399,30.615332,34.370216,CB,37.792024,2,,False
11189,Samuel,Thomas,68.701204,180.990665,29.245085,45.252424,39.528950,57.416118,56.859141,29.006325,...,46.454614,36.112015,28.538593,38.431084,33.396506,CB,37.639566,2,,False


In [5]:
for pos in players.Position.unique():
    if pos in ["QB_PP", "QB_DT"]:
        pos = "QB"
    
    schools[pos] = 0

In [6]:
posBalance = {
    "QB": 4,
    "RB": 4,
    "WR": 8,
    "TE": 5,
    "OL": 10,
    "DT": 6,
    "EDGE": 6,
    "LB": 7,
    "CB": 6,
    "S": 5,
    "K": 1,
    "P": 1,
}

## Recruiting

In [7]:
for i, row in players.iterrows():
    print("{}. {} {} ({}) - {}*".format(i, row["First"], row["Last"], row["Position"], row["Stars"]))

    pos = row["Position"]
    if pos in ["QB_PP", "QB_DT"]:
        pos = "QB"
    
    available = schools.loc[(schools.Total < 76) & (schools[pos] < posBalance[pos]+2)]
    # if at least a 3 star, recruit by probability
    
    if not available.empty:
        if row["Stars"] >= 3:
            s = rng.choice(available.Team, p=available[str(row["Stars"]) + "_Prob"] / available.sum()[str(row["Stars"]) + "_Prob"])

        # if a 2 star, assign randomly to a team that isn't full yet
        else:
            s = rng.choice(available.Team)
    
        print(s.upper())

        players.at[i, "School"] = s
        players.at[i, "Scholarship"] = True
        
        schools.loc[schools.Team==s, pos] += 1
        schools.loc[schools.Team == s, "Total"] += 1
        
    # check for all teams full
    vc = schools.Total.value_counts()
    if len(vc) == 1 and vc.index[0] == 76:
        break
        
    #print(vc.sort_index())
    print()

1. Andre Brown (OL) - 5*
TEXAS

2. Alijah Harris (DT) - 5*
AUBURN

3. Weston Jackson (DT) - 5*
LSU

4. Michael Finley (OL) - 5*
ALABAMA

5. Trevor Kennedy (OL) - 5*
NOTRE DAME

6. Daniel Kirk (OL) - 5*
LSU

7. John Hamilton (OL) - 5*
LSU

8. Michael Johnson (DT) - 5*
TEXAS

9. Maurice Mason (TE) - 5*
GEORGIA

10. Jonathan Sanchez (OL) - 5*
OHIO STATE

11. Mitchell Avila (OL) - 5*
PENN STATE

12. Matthew Fox (OL) - 5*
AUBURN

13. Joshua Jones (OL) - 5*
NOTRE DAME

14. Devin Vargas (OL) - 5*
TEXAS

15. Matthew Williams (CB) - 5*
OHIO STATE

16. Devin Simmons (LB) - 5*
FLORIDA

17. Jeremy Johnson (OL) - 5*
OREGON

18. Jay Henderson (OL) - 5*
TEXAS A&M

19. Reece Reid (OL) - 5*
AUBURN

20. Luke Perez (OL) - 5*
CLEMSON

21. Daniel Ruiz (OL) - 5*
ALABAMA

22. Kenny Perry (OL) - 5*
TEXAS

23. Nathaniel Herrera (OL) - 5*
LSU

24. Allyn Wright (OL) - 5*
ALABAMA

25. Dean Smith (TE) - 5*
OHIO STATE

26. Victor Jackson (EDGE) - 5*
ALABAMA

27. Jake Hoffman (DT) - 5*
LSU

28. Anthony Tucker (DT) -

NORTHWESTERN

220. Eric Rice (OL) - 4*
MISSOURI

221. Cole Carr (OL) - 4*
OKLAHOMA

222. Alexander Truong (QB_PP) - 4*
FLORIDA

223. Thomas Hail (LB) - 4*
LSU

224. Joseph Wilson (EDGE) - 4*
MISSISSIPPI STATE

225. Ethan Shepherd (S) - 4*
GEORGIA

226. Zachary Miller (K) - 4*
SOUTH CAROLINA

227. Josiah Adams (TE) - 4*
MICHIGAN STATE

228. Dawson Little (QB_PP) - 4*
TEXAS TECH

229. Isaac Harris (OL) - 4*
VIRGINIA

230. Thomas Daniels (OL) - 4*
MISSISSIPPI STATE

231. Kennan Smith (TE) - 4*
FLORIDA

232. Jonathan Davis (OL) - 4*
FLORIDA STATE

233. Yousef Porter (DT) - 4*
TEXAS

234. Jordan Strickland (TE) - 4*
OHIO STATE

235. Angel Vega (OL) - 4*
WASHINGTON

236. Nathan Brown (OL) - 4*
NOTRE DAME

237. Ryan Moore (OL) - 4*
VIRGINIA

238. Matthew David (LB) - 4*
LOUISIANA TECH

239. Bradley Dixon (EDGE) - 4*
FLORIDA ATLANTIC

240. Jacob Carpenter (QB_PP) - 4*
OKLAHOMA

241. Tyler Ellison (S) - 4*
GEORGIA

242. Lamont Coleman (RB) - 4*
EASTERN MICHIGAN

243. Jonathan Gomez (RB) - 4*
WA

423. Jarel Parra (QB_DT) - 4*
FLORIDA STATE

424. Alex Lawson (LB) - 4*
GEORGIA

425. Tyler Ramirez (CB) - 4*
FLORIDA STATE

426. Anthony Burnett (LB) - 4*
WISCONSIN

427. Nathan Smith (TE) - 4*
CALIFORNIA

428. Dakota Golden (QB_PP) - 4*
OREGON

429. Jerimiah Campbell (LB) - 4*
TEXAS

430. Brendan Hill (OL) - 4*
FLORIDA STATE

431. Colin Kennedy (RB) - 4*
MEMPHIS

432. Joshua Hunter (TE) - 4*
TENNESSEE

433. Israel Nunez (DT) - 4*
USC

434. Keagan Henry (K) - 4*
NORTHWESTERN

435. Gregory Perez (CB) - 4*
IOWA STATE

436. River Yang (WR) - 4*
MISSISSIPPI STATE

437. Mark Davis (S) - 4*
GEORGIA

438. Andrew Miranda (OL) - 4*
FLORIDA

439. Armando Charles (LB) - 4*
NEBRASKA

440. Carlos Medina (RB) - 4*
FLORIDA

441. Lucas Carr (RB) - 4*
ARIZONA STATE

442. Micheal Davis (OL) - 4*
AUBURN

443. Dylan Riley (LB) - 4*
GEORGIA

444. Ruben Peterson (LB) - 4*
ARIZONA STATE

445. Tyler Madden (LB) - 4*
OKLAHOMA STATE

446. Austin Johnson (OL) - 4*
PENN STATE

447. Jonathan Smith (OL) - 4*
WISCO


640. Tyree Edwards (OL) - 4*
WASHINGTON

641. Christian Walker (S) - 4*
OREGON

642. Kyle Ward (WR) - 4*
FLORIDA ATLANTIC

643. Jacob Bryant (WR) - 4*
UTEP

644. Brendan Watkins (LB) - 4*
EAST CAROLINA

645. Nathaniel Morales (DT) - 4*
MISSISSIPPI STATE

646. Moises Meyer (EDGE) - 4*
USC

647. Gerrod Richardson (OL) - 4*
SAN JOSE STATE

648. Tori Fisher (LB) - 4*
OLE MISS

649. Greyson Robbins (EDGE) - 4*
OREGON

650. Luis Dunn (LB) - 4*
VIRGINIA TECH

651. Tyler Miller (WR) - 4*
GEORGIA

652. Jonathan Kennedy (LB) - 4*
OHIO STATE

653. Hunter Randall (LB) - 4*
VIRGINIA

654. Gregory White (OL) - 4*
AUBURN

655. Paul Lopez (DT) - 4*
STANFORD

656. Matthew Stafford (LB) - 4*
MIAMI-FL

657. Nicholas Rogers (QB_DT) - 4*
CLEMSON

658. Mckenzie Travis (CB) - 4*
AUBURN

659. Abner Butler (QB_DT) - 4*
CLEMSON

660. Brandon Dodson (TE) - 4*
OREGON

661. Jonathan Jennings (LB) - 4*
TEXAS

662. John Oliver (OL) - 4*
GEORGIA TECH

663. Alejandro Chandler (DT) - 4*
AUBURN

664. Eduardo Washington


848. Jakob Chen (WR) - 4*
MICHIGAN

849. Vincent Noble (DT) - 4*
HAWAII

850. Michael Anderson (LB) - 4*
TEXAS A&M

851. Joshua Nava (WR) - 4*
WASHINGTON

852. Caleb Kelly (QB_PP) - 4*
BOISE STATE

853. Ronald Diaz (LB) - 4*
TEXAS TECH

854. Jeffrey Mccoy (P) - 4*
WASHINGTON

855. Nathaniel Blankenship (DT) - 4*
COLORADO

856. Edwin Martinez (QB_PP) - 4*
BAYLOR

857. Rodney Warner (LB) - 4*
KENTUCKY

858. Camron Lewis (S) - 4*
MARYLAND

859. Francesco Cabrera (LB) - 4*
OREGON

860. Joshua Kelly (RB) - 4*
WAKE FOREST

861. Nicholas James (EDGE) - 4*
UCLA

862. Luke Brown (TE) - 4*
MICHIGAN STATE

863. Daniel Montoya (WR) - 4*
MIDDLE TENNESSEE

864. Ryan Hahn (S) - 4*
MICHIGAN

865. Rafael Knight (OL) - 4*
OHIO STATE

866. Michael Kent (CB) - 4*
TEXAS

867. Daniel Li (QB_DT) - 4*
MINNESOTA

868. Seth Mccormick (LB) - 4*
OLE MISS

869. Brandon Cruz (WR) - 4*
GEORGIA

870. Corey Solis (OL) - 4*
MISSISSIPPI STATE

871. Liam Hughes (OL) - 4*
COLORADO

872. Jermaine Henry (RB) - 4*
OLE MISS


...


9235. Samuel Saunders (CB) - 2*
UL-MONROE

9236. Dylan Sullivan (WR) - 2*
AKRON

9237. Austin Underwood (LB) - 2*
EASTERN MICHIGAN

9238. Reed Andrews (EDGE) - 2*
MARSHALL

9239. Andrew Rush (TE) - 2*
UTAH STATE

9240. Casey Collins (RB) - 2*
HAWAII

9241. Cameron Dean (DT) - 2*
AIR FORCE

9242. Evan Flores (WR) - 2*
NAVY

9243. Ethan Lewis (WR) - 2*
NEW MEXICO STATE

9244. Jared Smith (WR) - 2*
BOWLING GREEN

9245. Ty Harmon (S) - 2*
NEW MEXICO

9246. Armando Mason (CB) - 2*
UTEP

9247. Robert Wiley (WR) - 2*
BALL STATE

9248. Caleb Garcia (TE) - 2*
GEORGIA SOUTHERN

9249. Raul Esparza (LB) - 2*
BALL STATE

9250. Shivam Crawford (LB) - 2*
MARSHALL

9251. Brandon Dillon (S) - 2*
WESTERN MICHIGAN

9252. Kenneth Wolfe (OL) - 2*
BYU

9253. Bradley Ortega (S) - 2*
CHARLOTTE

9254. Bryan Gonzalez (LB) - 2*
OLD DOMINION

9255. Matthew Jackson (S) - 2*
UTEP

9256. Jevon Castillo (RB) - 2*
COASTAL CAROLINA

9257. Bryan Mccarty (WR) - 2*
AKRON

9258. Luis Roberts (CB) - 2*
UAB

9259. Harrison 


9457. Andrew Yates (CB) - 2*
NEW MEXICO STATE

9458. Sultan Miller (TE) - 2*
EAST CAROLINA

9459. Trever Burgess (WR) - 2*

9460. Cristian Morales (QB_DT) - 2*
EASTERN MICHIGAN

9461. Jason Adkins (S) - 2*
GEORGIA SOUTHERN

9462. Muhammed Stafford (OL) - 2*
BALL STATE

9463. Steven Craig (OL) - 2*
MARSHALL

9464. Micheal Mcdaniel (EDGE) - 2*
MASSACHUSETTS

9465. Anthony Woods (TE) - 2*
COASTAL CAROLINA

9466. Benjamin Durham (WR) - 2*

9467. Brayden Parker (CB) - 2*
CHARLOTTE

9468. Samuel Wagner (QB_DT) - 2*
BALL STATE

9469. Shayden Beck (WR) - 2*

9470. Brett Kim (WR) - 2*

9471. Eduardo Nelson (WR) - 2*

9472. Chase Holmes (EDGE) - 2*
SAN JOSE STATE

9473. Noah Owens (WR) - 2*

9474. Michael Bush (DT) - 2*
OLD DOMINION

9475. Austin Li (QB_PP) - 2*
COASTAL CAROLINA

9476. Gerron Johnson (WR) - 2*

9477. Kejuan Walsh (OL) - 2*
MARSHALL

9478. Christian Wang (CB) - 2*
BUFFALO

9479. Dustin Munoz (S) - 2*
BOWLING GREEN

9480. Andres Smith (OL) - 2*
WYOMING

9481. John Mills (LB) - 2*


9671. Deryk Larson (S) - 2*
NEW MEXICO STATE

9672. Jose Jacobson (OL) - 2*
SOUTH ALABAMA

9673. Muneeb Yang (WR) - 2*

9674. John Esparza (CB) - 2*
SOUTH ALABAMA

9675. Brandon Fox (OL) - 2*
APPALACHIAN STATE

9676. Ryan Gordon (TE) - 2*
TEXAS STATE

9677. William Gonzalez (LB) - 2*
CENTRAL MICHIGAN

9678. Osama Webb (OL) - 2*
OLD DOMINION

9679. Jonathan Malone (OL) - 2*
APPALACHIAN STATE

9680. Ryan Blair (DT) - 2*
RICE

9681. Richard Murray (TE) - 2*
TEXAS STATE

9682. Kendrick Mcdonald (WR) - 2*

9683. Caleb Watson (DT) - 2*
KENT STATE

9684. Logan Gutierrez (S) - 2*
SOUTH ALABAMA

9685. Hassan Fisher (LB) - 2*
NEW MEXICO

9686. Caleb Larson (WR) - 2*

9687. Richard Davis (RB) - 2*
GEORGIA SOUTHERN

9688. Vincent Byrd (OL) - 2*
NAVY

9689. Richard Griffin (EDGE) - 2*
SOUTH ALABAMA

9690. Larry Davis (DT) - 2*
BALL STATE

9691. Tyler Sanchez (CB) - 2*
RICE

9692. Devin Brown (LB) - 2*
SOUTH ALABAMA

9693. Owen Smith (S) - 2*
NEW MEXICO STATE

9694. Tyler Jackson (S) - 2*
NEW MEXIC


9900. Qwinton Mclaughlin (OL) - 2*
NAVY

9901. Joseph Choi (DT) - 2*
NEW MEXICO STATE

9902. Andrew Watts (OL) - 2*
OHIO

9903. Andrew Matthews (OL) - 2*
OHIO

9904. Jordan Hensley (OL) - 2*
OLD DOMINION

9905. Grant Moore (S) - 2*

9906. Zachary Anderson (DT) - 2*
CHARLOTTE

9907. Danny Martinez (WR) - 2*

9908. Eduardo Peterson (OL) - 2*
UL-MONROE

9909. Jared Reed (CB) - 2*

9910. Tyler Davidson (TE) - 2*

9911. Daniel Chase (DT) - 2*
UTEP

9912. Carlos Wells (WR) - 2*

9913. Christian Perez (EDGE) - 2*
AKRON

9914. Semaj Walters (S) - 2*

9915. Hayden Golden (OL) - 2*
NAVY

9916. James Powell (P) - 2*

9917. Kyler Francis (DT) - 2*
MIAMI-OH

9918. Isaiah Mack (CB) - 2*

9919. Isaac Steele (CB) - 2*

9920. Brian Pham (WR) - 2*

9921. Christopher Williams (WR) - 2*

9922. Garrett Adams (WR) - 2*

9923. Jordan Tucker (EDGE) - 2*
BOWLING GREEN

9924. Marvin Wright (EDGE) - 2*
BUFFALO

9925. Nicholas Callahan (WR) - 2*

9926. Matthew Jones (LB) - 2*
OLD DOMINION

9927. Connar Welch (DT


10134. Michael Delgado (K) - 2*

10135. Daniel Bernal (OL) - 2*

10136. Alexander Collier (TE) - 2*

10137. Brett West (TE) - 2*

10138. Darius Brooks (WR) - 2*

10139. Austin Smith (OL) - 2*

10140. Richard Rodriguez (WR) - 2*

10141. Brandon Fitzgerald (WR) - 2*

10142. Noah Smith (OL) - 2*

10143. Ryan Garcia (WR) - 2*

10144. Sterling Rodriguez (EDGE) - 2*
NEW MEXICO STATE

10145. Dalton Rodriguez (CB) - 2*

10146. Jaylon Santana (OL) - 2*

10147. Casey Rollins (EDGE) - 2*
NEW MEXICO STATE

10148. David Evans (EDGE) - 2*
SOUTH ALABAMA

10149. Tyler Garcia (S) - 2*

10150. Medin Zuniga (WR) - 2*

10151. Steven Munoz (TE) - 2*

10152. Eric Valenzuela (DT) - 2*
LIBERTY

10153. Izak Rodriguez (WR) - 2*

10154. Jason Johnson (WR) - 2*

10155. Joe Williams (P) - 2*

10156. Zachary Gonzalez (WR) - 2*

10157. Mikael Martin (OL) - 2*

10158. Aaron Mitchell (P) - 2*

10159. Taylor Bryant (WR) - 2*

10160. Miles Zavala (QB_PP) - 2*
LIBERTY

10161. Benjamin Peterson (K) - 2*

10162. Marcos Ca

## Recruiting Summarization

In [8]:
players[["First", "Last", "Height", "Weight", "Position", "P_BEST", "Stars", "School"]].head(30)

,First,Last,Height,Weight,Position,P_BEST,Stars,School
1,Andre,Brown,77.785520,370.251049,OL,108.370395,5,Texas
2,Alijah,Harris,76.678068,299.611678,DT,103.209994,5,Auburn
3,Weston,Jackson,78.264296,344.850574,DT,102.063971,5,LSU
4,Michael,Finley,80.998230,370.025541,OL,101.260238,5,Alabama
5,Trevor,Kennedy,78.825032,337.558931,OL,101.244444,5,Notre Dame
6,Daniel,Kirk,77.510994,332.242557,OL,101.083485,5,LSU
7,John,Hamilton,77.416660,317.407020,OL,100.908430,5,LSU
8,Michael,Johnson,77.856353,269.236900,DT,100.123121,5,Texas
9,Maurice,Mason,73.428588,190.292214,TE,100.007722,5,Georgia
10,Jonathan,Sanchez,77.127234,319.586988,OL,99.011888,5,Ohio State


In [9]:
schools[["Team"] + ["QB"] + [pos for pos in players.Position.unique() if pos not in ["QB_PP", "QB_DT"]]]

,Team,QB,OL,DT,TE,CB,LB,EDGE,K,S,WR,P,RB
0,Alabama,6,12,7,7,6,9,8,2,7,6,1,5
1,Georgia,4,12,8,7,8,9,5,0,7,10,1,5
2,LSU,5,12,8,7,7,5,7,3,7,9,1,5
3,Oklahoma,6,12,8,7,6,9,8,1,3,9,2,5
4,Ohio State,4,12,8,7,8,9,7,1,6,6,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,New Mexico State,2,12,3,7,8,8,7,3,7,10,3,6
126,Texas State,1,11,8,7,8,4,8,3,7,10,3,6
127,UL-Monroe,6,11,6,7,8,5,6,1,7,10,3,6
128,Coastal Carolina,3,11,7,7,8,4,7,3,7,10,3,6


## Add Walkons to fill out rosters

In [10]:
walkons = players.loc[players.School == ""]
walkons

,First,Last,Height,Weight,T_Strength,T_Athleticism,T_Dexterity,T_IQ,T_Personality,T_Arm,...,S_ReadOff,S_KickPow,S_KickAcc,S_PuntPow,S_PuntAcc,Position,P_BEST,Stars,School,Scholarship
9395,Thomas,Jackson,74.685089,213.735536,31.652807,63.289688,61.071520,51.486455,49.452538,29.938917,...,49.606341,38.971177,53.834982,38.566876,33.442119,WR,56.252401,2,,False
9397,Deonte,Snyder,70.642182,170.088776,28.716597,71.947667,70.854587,34.164477,37.158924,28.772093,...,23.895422,38.863885,41.982309,46.874697,31.945095,WR,56.249140,2,,False
9413,Angel,King,74.175024,220.212292,31.424329,67.940886,51.378442,45.301790,35.262724,46.000213,...,38.057255,38.770499,55.643994,43.105760,59.379388,WR,56.186179,2,,False
9420,Brandon,Johnson,75.457221,194.442962,12.901717,59.362070,85.999032,54.016502,20.437162,34.667949,...,54.312162,41.041988,42.978087,41.095249,53.743983,WR,56.161735,2,,False
9421,Brandon,Freeman,70.244179,171.243196,23.188949,69.703414,65.187017,61.681932,38.699514,31.886006,...,49.658941,30.327755,22.431135,23.902007,37.634649,WR,56.161678,2,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11186,Alexander,Taylor,73.069718,192.576496,12.249627,41.622411,81.585046,51.615538,53.051356,17.944516,...,54.973832,27.942084,36.691033,32.014368,42.943702,WR,38.044633,2,,False
11187,Ryan,Anderson,73.656113,191.024157,32.852587,48.698064,52.321087,44.950419,43.752979,33.823738,...,49.772971,22.623272,31.602734,24.885373,45.944774,EDGE,37.891980,2,,False
11188,Jose,Taylor,70.901010,198.336853,26.064285,44.826046,63.228263,55.632430,51.818022,15.915664,...,57.518376,31.744278,35.300399,30.615332,34.370216,CB,37.792024,2,,False
11189,Samuel,Thomas,68.701204,180.990665,29.245085,45.252424,39.528950,57.416118,56.859141,29.006325,...,46.454614,36.112015,28.538593,38.431084,33.396506,CB,37.639566,2,,False


In [11]:
posCounts = players.groupby(["School", "Position"]).agg("count")["Stars"]

for school in schools.Team.sample(frac=1):
    print(school)
    pc = posCounts.loc[school]
    print(pc)
    
    for pos in posBalance:
        if pos not in pc and pos != "QB":
            c = 0
        elif pos == "QB":
            if "QB_DT" in pc:
                dt = pc["QB_DT"]
            else:
                dt = 0
            
            if "QB_PP" in pc:
                pp = pc["QB_PP"]
            else:
                pp = 0
            
            c = dt + pp
        else:
            c = pc[pos]
        
        if c < posBalance[pos]:
            print("Too few {}s! Need {} more.".format(pos, posBalance[pos] - c))
        
        while c < posBalance[pos]-1:
            if pos != "QB":
                bestWOs = walkons.loc[walkons.Position == pos]
            else:
                bestWOs = walkons.loc[(walkons.Position == "QB_DT") | (walkons.Position == "QB_PP")]
            
            if not bestWOs.empty:
                p = players.loc[bestWOs.index[0]]
                players.at[p.name, "School"] = school
                walkons = walkons.drop(index=p.name)
                print("Signed {}. {} {} as Walkon.".format(p.name, p.First, p.Last))
                c += 1
            else:
                print("No recruits left at that position. We'll make do without.")
                break
    
    print()


Toledo
Position
CB        6
DT        8
EDGE      7
K         2
LB        9
OL        9
P         1
QB_DT     2
QB_PP     2
RB        6
S         7
TE        7
WR       10
Name: Stars, dtype: int64
Too few OLs! Need 1 more.

UTEP
Position
CB        8
DT        4
EDGE      4
K         3
LB        7
OL       12
P         3
QB_DT     3
QB_PP     2
RB        6
S         7
TE        7
WR       10
Name: Stars, dtype: int64
Too few DTs! Need 2 more.
Signed 10188. Jason Martin as Walkon.
Too few EDGEs! Need 2 more.
Signed 10182. Tyler Richards as Walkon.

Syracuse
Position
CB        8
DT        6
EDGE      8
K         3
LB        8
OL       11
P         1
QB_PP     5
RB        6
S         5
TE        7
WR        8
Name: Stars, dtype: int64

Akron
Position
CB        8
DT        5
EDGE      7
K         3
LB        4
OL       12
P         3
QB_PP     4
RB        6
S         7
TE        7
WR       10
Name: Stars, dtype: int64
Too few DTs! Need 1 more.
Too few LBs! Need 3 more.
Signed 10200. Joel M


Boise State
Position
CB        8
DT        8
EDGE      6
K         2
LB        8
OL       10
P         2
QB_DT     2
QB_PP     4
RB        5
S         7
TE        7
WR        7
Name: Stars, dtype: int64
Too few WRs! Need 1 more.

Wake Forest
Position
CB        8
DT        5
EDGE      4
K         3
LB        9
OL       11
P         3
QB_DT     1
QB_PP     3
RB        6
S         7
TE        6
WR       10
Name: Stars, dtype: int64
Too few DTs! Need 1 more.
Too few EDGEs! Need 2 more.
Signed 10444. Darius Alvarez as Walkon.

Appalachian State
Position
CB        8
DT        5
EDGE      4
K         3
LB        9
OL       12
P         3
QB_DT     1
QB_PP     1
RB        6
S         7
TE        7
WR       10
Name: Stars, dtype: int64
Too few QBs! Need 2 more.
Signed 10364. Ethan Hickman as Walkon.
Too few DTs! Need 1 more.
Too few EDGEs! Need 2 more.
Signed 10458. Jerry Parra as Walkon.

Arizona
Position
CB        8
DT        3
EDGE      7
K         3
LB        7
OL       11
P         3
QB_D

Signed 10644. Eric Wang as Walkon.
Too few LBs! Need 3 more.
Signed 10904. Victor Perez as Walkon.
Signed 10912. Luis Mendez as Walkon.

Liberty
Position
CB        8
DT        7
EDGE      3
K         3
LB        6
OL       12
P         3
QB_PP     4
RB        6
S         7
TE        7
WR       10
Name: Stars, dtype: int64
Too few EDGEs! Need 3 more.
Signed 10679. Kameryn Russell as Walkon.
Signed 10683. Bradley Scott as Walkon.
Too few LBs! Need 1 more.

Arkansas
Position
CB        8
DT        4
EDGE      6
K         2
LB        9
OL       12
P         3
QB_DT     4
RB        6
S         7
TE        5
WR       10
Name: Stars, dtype: int64
Too few DTs! Need 2 more.
Signed 10366. Matthew Lopez as Walkon.

Georgia Southern
Position
CB        8
DT        1
EDGE      4
K         3
LB        9
OL       12
P         3
QB_DT     4
QB_PP     2
RB        6
S         7
TE        7
WR       10
Name: Stars, dtype: int64
Too few DTs! Need 5 more.
Signed 10372. Kevin Holmes as Walkon.
Signed 10421. J

## Jersey Numbers

In [12]:
# todo

## Scale Age

In [13]:
players = players.loc[players.School != ""]

In [14]:
players["Age"] = 0
players["Age"] = players["Age"].apply(lambda x: rng.choice(range(4)))

/home/zaned/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/zaned/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
for c in players.columns:
    if "S_" in c:
        players[c] *= (0.75 + players["Age"]*0.0833)

/home/zaned/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Convert School Names

In [16]:
for s in schools.Team.sort_values():
    print(s)

Air Force
Akron
Alabama
Appalachian State
Arizona
Arizona State
Arkansas
Arkansas State
Army
Auburn
BYU
Ball State
Baylor
Boise State
Boston College
Bowling Green
Buffalo
California
Central Florida
Central Michigan
Charlotte
Cincinnati
Clemson
Coastal Carolina
Colorado
Colorado State
Connecticut
Duke
East Carolina
Eastern Michigan
Florida
Florida Atlantic
Florida International
Florida State
Fresno State
Georgia
Georgia Southern
Georgia State
Georgia Tech
Hawaii
Houston
Illinois
Indiana
Iowa
Iowa State
Kansas
Kansas State
Kent State
Kentucky
LSU
Liberty
Louisiana Tech
Louisville
Marshall
Maryland
Massachusetts
Memphis
Miami-FL
Miami-OH
Michigan
Michigan State
Middle Tennessee
Minnesota
Mississippi State
Missouri
NC State
Navy
Nebraska
Nevada
New Mexico
New Mexico State
North Carolina
North Texas
Northern Illinois
Northwestern
Notre Dame
Ohio
Ohio State
Oklahoma
Oklahoma State
Old Dominion
Ole Miss
Oregon
Oregon State
Penn State
Pittsburgh
Purdue
Rice
Rutgers
SMU
San Diego State
San Jose

In [17]:
converter = {
    "BYU": "Brigham Young",
    "Miami-FL": "Miami (FL)",
    "Miami-OH": "Miami (OH)",
    "SMU": "Southern Methodist",
    "Central Florida": "UCF",
    "UL-Monroe": "Louisiana-Monroe",
    "UL-Lafayette": "Louisiana"
}

In [18]:
def ConvertName(n):
    if n in converter:
        return converter[n]
    else:
        return n

## Final Output

In [19]:
players

,First,Last,Height,Weight,T_Strength,T_Athleticism,T_Dexterity,T_IQ,T_Personality,T_Arm,...,S_KickPow,S_KickAcc,S_PuntPow,S_PuntAcc,Position,P_BEST,Stars,School,Scholarship,Age
1,Andre,Brown,77.785520,370.251049,96.205956,38.034630,31.360394,33.230310,49.318560,43.095424,...,48.532710,59.505983,49.065119,54.648812,OL,108.370395,5,Texas,True,1
2,Alijah,Harris,76.678068,299.611678,90.896314,49.165386,29.255212,74.229956,43.171610,42.392602,...,33.313661,37.825035,35.067671,44.738782,DT,103.209994,5,Auburn,True,1
3,Weston,Jackson,78.264296,344.850574,94.834343,34.736893,20.231105,57.028000,48.219405,31.305556,...,30.687106,31.501133,33.360733,30.193036,DT,102.063971,5,LSU,True,1
4,Michael,Finley,80.998230,370.025541,80.890109,28.750672,23.364785,22.856387,50.052748,65.370502,...,72.078279,64.604403,74.132828,49.749764,OL,101.260238,5,Alabama,True,3
5,Trevor,Kennedy,78.825032,337.558931,92.124447,29.626324,25.696194,41.070697,57.389817,46.083700,...,35.203844,28.017668,36.217114,34.619237,OL,101.244444,5,Notre Dame,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10912,Luis,Mendez,72.968641,194.842719,43.397402,53.698355,61.833605,62.864089,48.856689,16.039461,...,41.023556,30.147922,42.200343,49.742598,LB,49.134355,2,Miami-OH,False,3
10947,Jermiah,Lynch,71.557363,220.200763,32.812967,58.650870,48.228699,53.012840,36.217638,42.765180,...,50.288033,36.985531,43.841623,36.314702,LB,48.706334,2,Bowling Green,False,2
10971,Moises,Thompson,71.648371,218.226612,37.112360,58.810433,42.482008,44.525039,42.685321,25.204157,...,39.797014,24.383046,35.480213,42.102787,LB,48.298654,2,Coastal Carolina,False,1
11021,Jacob,Perez,76.098157,232.452969,36.891290,55.897072,31.995246,66.502126,47.181180,12.664894,...,25.339267,44.026613,26.907445,32.733693,LB,47.639433,2,Coastal Carolina,False,0


In [20]:
# todo: save to database

#p = models.Player(first="Spencer", last="Rattler", jersey=1, age="SO", school=models.School.objects.get(name="Oklahoma"))
#p.save()

In [21]:
def SavePlayerSegment(start):
    for i, r in players.loc[start:start+999].iterrows():
        #print(i)

        f = r["First"]
        l = r["Last"]
        h = r["Height"]
        w = r["Weight"]

        p = r["Position"]
        if p in ["QB_DT", "QB_PP"]:
            p = "QB"
        
        a = ("FR", "SO", "JR", "SR")[r["Age"]]
        s = models.School.objects.get(name=ConvertName(r["School"]))
        schol = r["Scholarship"]
        stars = r["Stars"]
        
        t_strength = r["T_Strength"]  # unnecessary; remove when cleaning up code
        t_athleticism = r["T_Athleticism"]
        t_dexterity = r["T_Dexterity"]
        t_iq = r["T_IQ"]
        t_personality = r["T_Personality"]
        t_arm = r["T_Arm"]
        t_leg = r["T_Leg"]

        p = models.Player(first=f, last=l, jersey=1, height=h, weight=w,
                          position=p, age=a, school=s, scholarship=schol, stars=stars,
                          t_strength=t_strength, t_athleticism=t_athleticism, t_dexterity=t_dexterity,
                          t_iq=t_iq, t_personality=t_personality, t_arm=t_arm, t_leg=t_leg,
                          s_armStr=r["S_ArmStr"], s_armAccShort=r["S_ArmAccShort"], s_armAccMed=r["S_ArmAccMed"], s_armAccLong=r["S_ArmAccLong"],
                          s_readDefPass=r["S_ReadDefPass"], s_readDefRun=r["S_ReadDefRun"],
                          s_speed=r["S_Speed"], s_acceleration=r["S_Acceleration"], s_agility=r["S_Agility"],
                          s_runPower=r["S_RunPower"], s_vertical=r["S_Vertical"], s_hands=r["S_Hands"],
                          s_runBlock=r["S_RunBlock"], s_passBlock=r["S_PassBlock"],
                          s_tackApproach=r["S_TackApproach"], s_tackBringDown=r["S_TackBringDown"], s_shedBlockStr=r["S_ShedBlockStr"], s_shedBlockAgl=r["S_ShedBlockAgl"],
                          s_coverMan=r["S_CoverMan"], s_coverZone=r["S_CoverZone"], s_readOff=r["S_ReadOff"],
                          s_kickPow=r["S_KickPow"], s_kickAcc=r["S_KickAcc"],
                          s_puntPow=r["S_PuntPow"], s_puntAcc=r["S_PuntAcc"])
        p.save()
SavePlayerSegment(1)

In [22]:
SavePlayerSegment(1001)

In [23]:
SavePlayerSegment(2001)

In [24]:
SavePlayerSegment(3001)

In [25]:
SavePlayerSegment(4001)

In [26]:
SavePlayerSegment(5001)

In [27]:
SavePlayerSegment(6001)

In [28]:
SavePlayerSegment(7001)

In [29]:
SavePlayerSegment(8001)

In [30]:
SavePlayerSegment(9001)

In [31]:
SavePlayerSegment(10001)

In [32]:
SavePlayerSegment(11001)

In [33]:
assert False  # so delete won't be run automatically

AssertionError: 

In [ ]:
for p in models.Player.objects.all():
    p.delete()

In [ ]:
players.columns